In [1]:
# from fastai.imports import *
from fastai.structured import *

ModuleNotFoundError: No module named 'cv2'

Note : in the imports.core.py file, unhighlight # 29-31, 24 and download fastprogress

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import re
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

sns.set_style('darkgrid')

In [ ]:
file = '../data/bluebook-for-bulldozers/Train.csv'

In [ ]:
df_raw = pd.read_csv(file, parse_dates=['saledate'], low_memory=False)

In [ ]:
df_raw.head()

In [ ]:
df_raw.info()

In [ ]:
df_raw.describe()

In [ ]:
df_raw['SalePrice'] = np.log(df_raw['SalePrice'])

In [ ]:
X, y = df_raw.drop('SalePrice', axis=1), df_raw['SalePrice'].copy()

In [ ]:
rf = RandomForestRegressor(max_samples=10000).fit(X, y)

In [ ]:
# extracting features fromthe date attribute
def add_date(df, name):
    fld = df[name]
    target = re.sub('[Dd]ate$', '', name)
    for n in ['Year', 'Month', 'Day']:
        df[target+n] = getattr(fld.dt, n.lower())
    df.drop(name, axis=1, inplace=True)
add_date(df_raw, 'saledate')

In [ ]:
df_raw

In [ ]:
df_raw['UsageBand'] = pd.Categorical(df_raw['UsageBand']).codes+1
df_raw['UsageBand']

In [ ]:
df_raw.isnull().sum()

In [ ]:
# Changing object to categorical data types
for column, content in df_raw.items():
    if not pd.api.types.is_numeric_dtype(content):
        df_raw[column] = content.astype('category').cat.as_ordered()

In [ ]:
# dealing with null values and filling the the values of missing numeric type with their median value       
for column, content in df_raw.items():
    if pd.api.types.is_numeric_dtype(content):
        if pd.isnull(content).sum():                         
            df_raw[column+'_na'] = pd.isnull(content)
            df_raw[column] = content.fillna(content.median())

In [ ]:
df_raw

In [ ]:
# Dealing with categorical values
for column, content in df_raw.items():
    if not pd.api.types.is_numeric_dtype(content):
        if pd.isnull(content).sum():                                     
            df_raw[column+'_na'] = pd.isnull(content)
            df_raw[column] = pd.Categorical(content).codes+1

In [ ]:
df_raw

In [ ]:
display(df_raw.iloc[4:, 9::2].tail(20).T)

In [ ]:
df_raw.info()

In [ ]:
X, y = df_raw.drop('SalePrice', axis=1), df_raw['SalePrice'].copy()

In [ ]:
rf = RandomForestRegressor(max_samples=10000, n_jobs=-1).fit(X, y)
rf.score(X, y)

##  Splitting the data into train and validation sets

In [ ]:
test_size = 12000
train_size = len(X) - test_size

def split_data(df, size):
    #splits data in train and validation sets
    return df[:size].copy(), df[size:].copy()

In [ ]:
X_train, X_valid = split_data(X, train_size)
y_train, y_test = split_data(y, train_size)

In [ ]:
X_train.shape, X_valid.shape, y_train.shape

In [ ]:
rf= RandomForestRegressor(max_samples=20000, n_jobs=-1).fit(X_train, y_train)
rf.score(X_train, y_train)

In [ ]:
X_trn, X_tst, y_trn, y_tst = train_test_split(X, y, test_size=0.30)

In [ ]:
rf= RandomForestRegressor(max_samples=10000, n_jobs=-1).fit(X_trn, y_trn)
rf.score(X_trn, y_trn)

In [ ]:
import os
os.makedirs('tmp', exits_ok=True)
df_raw.to_feather('tmp/bulldozer-raw1')